In [2]:
# loading things
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
from scipy.optimize import minimize

In [3]:
def sigmoid(X):
    """
    Compute the sigmoid function
    """
    denominator = 1.0 + np.exp(-1.0 * X)
    return 1.0 / denominator

In [4]:
def sigmoid_gradient(a):
    """
    computes the gradient of the sigmoid function at input value z
    """
    a = np.array(a)
    output_shape = a.shape
    output = np.array([a * (1 - a)])
    output.reshape(output_shape)
    return output

In [5]:
def square_up(theta_vector, input_layer_size, hidden_layer_size):
    """
    Returns the theta vector to matrices
    """
    vector_1_length = (input_layer_size + 1) * hidden_layer_size
    matrix_1 = theta_vector[0:vector_1_length].reshape((hidden_layer_size, input_layer_size + 1))
    matrix_2 = theta_vector[vector_1_length:]
    matrix_2 = matrix_2.reshape((output_layer_size, hidden_layer_size + 1))
    return (matrix_1, matrix_2)

In [6]:
def flatten_out(thetas):
    """
    Converts features from matrices to vectors.
    """
    return np.hstack((thetas[0].reshape(-1), thetas[1].reshape(-1)))

In [7]:
def forward_propogation(theta1, theta2, X):
    """
    Forward propogation for a three layer nn
    Inputs:
        i = input layer
        h = hidden layer
        o = output layer (number of classes)
        m = number of training sets / observations
        theta1: (i+1) x h numpy array
        theta2: (h+1) x o numpy array
        X: m x i numpy array
    
    """
    # input layer
    m, n = X.shape
    a1 = np.ones((m, n+1))
    a1[:,1:] = X
    # hidden Layer
    z2 = np.dot(theta1, a1.T)
    a2_0 = sigmoid(z2)
    a2 = np.ones((a2_0.shape[0]+1, m))
    a2[1:,:] = a2_0
    # output layer
    z3 = np.dot(theta2, a2)
    a3 = sigmoid(z3)    
    return a1, z2, a2, z3, a3

In [8]:
def cost_function(thetas, m, X, Y, input_layer_size, hidden_layer_size, output_layer_size):
    """
    Calculates the cost function J after a round of forward propogation
    Inputs
        i = input layer
        h = hidden layer
        o = output layer (number of classes)
        m = number of training sets / observations
        theta1: (i+1) x h numpy array
        theta2: (h+1) x o numpy array
        X: m x i numpy array
    Output is a float
    """
    t = square_up(thetas, input_layer_size, hidden_layer_size)
    a1, z1, a2, z2, a3 = forward_propogation(t[0], t[1], X)
    h = a3.T
    h_r = h.reshape(-1)
    y = np.eye(10)[Y-1][:,0,:]
    y_r = y.reshape(-1)
    j1 = np.dot(y_r, np.log(h_r))
    j2 = np.dot((1-y_r), np.log(1 - h_r))
    J = (-1./len(Y)) * (j1+j2)
    return J

In [9]:
def cost_function_reg(thetas, lamda, m, X, Y, input_layer_size, hidden_layer_size, output_layer_size):
    """
    Calculates the regularized cost function J after a round of forward propogation
    Inputs
        thetas are rolled feature spaces
        lamda is a float
        i = input layer
        h = hidden layer
        o = output layer (number of classes)
        m = number of training sets / observations
        theta1: (i+1) x h numpy array
        theta2: (h+1) x o numpy array
        X: m x i numpy array
    Output is a float
    """
    base = cost_function(thetas, X, Y, input_layer_size, hidden_layer_size, output_layer_size)
    reg_0 = lamda / float(2*m)
    reg_1 = sum([t**2 for t in thetas])
    reg_term = reg_0 * reg_1
    return base + reg_term

In [10]:
def cost_gradient(thetas, m, X, Y, input_layer_size, hidden_layer_size, output_layer_size):
    """
    Approximates the gradient vector of the NN via backpropagation
    """
    init1_shape = (hidden_layer_size, input_layer_size+1)
    init2_shape = (output_layer_size, hidden_layer_size+1)
    delta_1 = np.zeros(init1_shape)
    delta_2 = np.zeros(init2_shape)
    count = 0
    t = square_up(thetas, input_layer_size, hidden_layer_size)
    y = np.eye(10)[Y-1][:,0,:]
    ## back propagation
    for x in X:
        a1, z2, a2, z3, a3 = forward_propogation(t[0], t[1], x.reshape(1, input_layer_size))
        # layer three
        # layer three
        y = np.zeros(10)
        y[Y[count]-1]=1
        delta_3_k = (a3.T - y)
        delta_3_k = delta_3_k.T
        # layer two
        term1 = np.dot(t[1].T, delta_3_k) 
        term2 = sigmoid_gradient(a2)[0]
        delta_2_k = term1 * term2
        delta_2_k = delta_2_k[1:]
        # calculating delta terms
        term_2_ij = np.dot(delta_3_k, a2.T)
        term_1_ij = np.dot(delta_2_k, a1)
        delta_2 = delta_2 + term_2_ij
        delta_1 = delta_1 + term_1_ij
        count+=1

    delta_1 = delta_1 / float(m)
    delta_2 = delta_2 / float(m)
    deltas = flatten_out((delta_1, delta_2))
    return deltas

In [11]:
def cost_gradient_reg(thetas, lamda, m, X, Y, input_layer_size, hidden_layer_size, output_layer_size):
    """
    Approximates the regularised gradient vector of the NN via backpropagation
    
    """
    grad = cost_gradient(thetas, m, X, Y, input_layer_size, hidden_layer_size, output_layer_size)
    grad = square_up(grad, input_layer_size, hidden_layer_size)
    term1 = lamda / m
    thetas = square_up(thetas, input_layer_size, hidden_layer_size)
    t1 = np.zeros(thetas[0].shape)
    t1[:,1:] = thetas[0][:,1:]
    t2 = np.zeros(thetas[1].shape)
    t2[:,1:] = thetas[1][:,1:]
    grad1 = grad[0] + (term1 * t1)
    grad2 = grad[1] + (term1 * t2)
    grad = flatten_out((grad1, grad2))
    return grad

In [12]:
def numerical_gradient(theta1, theta2, lamda, m, X, Y, input_layer_size, hidden_layer_size, output_layer_size):
    """
    Approximates the gradient of the cost function by perturbing 
    element ij of layer ell by eps
    """
    thetas = flatten_out((theta1, theta2))
    f = []
    eps_g = 1e-4
    for i in range(len(thetas)):
        print "Element", i, "of", len(thetas)
        theta_high = thetas
        theta_low = thetas
        theta_high[i] = theta_high[i] + eps_g
        theta_low[i] = theta_low[i] - eps_g
        j_high = cost_function_reg(theta_high, lamda, m, X, Y, input_layer_size, hidden_layer_size, output_layer_size)
        j_low = cost_function_reg(theta_low, lamda, m, X, Y, input_layer_size, hidden_layer_size, output_layer_size)
        f_theta_approx = (j_high - j_low) / (2 * eps_g)
        f.append(f_theta_approx[0])
    return np.array(f)

In [13]:
#f_numerical = numerical_gradient(theta1, theta2, lamda, m, X, Y, y_large, input_layer_size, hidden_layer_size, output_layer_size)

In [14]:
# loading features
theta = loadmat('ex3weights.mat')
theta1 = theta['Theta1']
theta2 = theta['Theta2']
# loading training set
dta = loadmat('ex3data1.mat')
X = dta['X']
Y = dta['y']
m, n = X.shape 
#Y[Y==10] = 0

input_layer_size = X.shape[1]
hidden_layer_size = theta1.shape[0]
output_layer_size = theta2.shape[0]

lamda = 1.
eps_init = 0.12
init1 = np.random.rand(hidden_layer_size, input_layer_size+1) * 2 * eps_init - eps_init
init2 = np.random.rand(output_layer_size, hidden_layer_size+1) * 2 * eps_init - eps_init
thetas_0 = flatten_out((init1, init2))
thetas = flatten_out((theta1, theta2))
print thetas.shape
theta_opt = minimize(
    cost_function, x0=thetas_0, 
    args=(m, X, Y, input_layer_size, hidden_layer_size, output_layer_size), 
    method="TNC", jac=cost_gradient, 
    options={"maxiter":500, "disp":True}).x
theta_opt = square_up(theta_opt, input_layer_size, hidden_layer_size)

(10285,)


/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:21: RuntimeWarning: divide by zero encountered in log


In [246]:
def prediction(a, Y):
    y = np.eye(10)[Y-1][:,0,:]
    pr = np.argmax(a.T, axis=1).reshape(Y.shape)
    accuracy = 100 * sum((pr==(Y-1))) / float(len(Y))
    print 'The neural network correctly predicts %f percent of the cells' % accuracy
    return

In [247]:
prediction(a3, Y)

The neural network correctly predicts 99.940000 percent of the cells
